# Investment Research Agent - Multi-Agent System (Skeleton)

## Project Overview
This notebook implements an autonomous Investment Research Agent that demonstrates:

## Architecture
- **Multi-Agent System**: Coordinator, Specialist Agents (News, Technical, Fundamental)
- **Memory System**: FAISS vector database for persistent learning
- **Data Sources**: Yahoo Finance, NewsAPI, FRED, Alpha Vantage
- **Interface**: Gradio web interface for user interaction

## 1. Environment Setup and Dependencies

In [2]:
# Install required packages
!pip install langchain langchain-openai langchain-community yfinance pandas numpy matplotlib seaborn plotly gradio faiss-cpu python-dotenv requests fredapi newsapi-python chromadb

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/949.2 kB ? eta -:--:--
     ---------------------------------------- 0.0/949.2 kB ? eta -:--:--
     ----------- ---------------------------- 262.1/949.2 kB ? eta -:--:--
     -------------------- ----------------- 524.3/949.2 kB 1.1 MB/s eta 0:00:01
     ----------------------------- ------ 786.4/949.2 kB 932.9 kB/s eta 0:00:01
     -------------------------------------- 949.2/949.2 kB 1.0 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Get

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyppeteer 2.0.0 requires websockets<11.0,>=10.0, but you have websockets 15.0.1 which is incompatible.


In [4]:
# Import required libraries
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime, timedelta
import warnings
import logging
import time
from typing import Dict, List, Any, Optional
import gradio as gr

# LangChain imports
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain.tools import BaseTool, tool
from langchain_openai import AzureChatOpenAI
from langchain.memory import ConversationBufferWindowMemory
from langchain.schema import BaseMessage, HumanMessage, AIMessage
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.vectorstores import FAISS
from langchain_openai import AzureOpenAIEmbeddings
from langchain.docstore.document import Document

# Data source imports
import yfinance as yf
from newsapi import NewsApiClient
from fredapi import Fred
import requests

# Environment variables
from dotenv import load_dotenv
load_dotenv()

warnings.filterwarnings('ignore')
logging.basicConfig(level=logging.INFO)

In [ ]:
# Configuration from environment variables
AZURE_OPENAI_API_KEY = os.getenv('AZURE_OPENAI_API_KEY')
AZURE_OPENAI_ENDPOINT = os.getenv('AZURE_OPENAI_ENDPOINT')
AZURE_OPENAI_GPT_DEPLOYMENT_NAME = os.getenv('AZURE_OPENAI_GPT_DEPLOYMENT_NAME')
AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME = os.getenv('AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME')
AZURE_OPENAI_API_VERSION = os.getenv('AZURE_OPENAI_API_VERSION', '2024-02-15-preview')

ALPHA_VANTAGE_API_KEY = os.getenv('ALPHA_VANTAGE_API_KEY')
NEWS_API_KEY = os.getenv('NEWSAPI_KEY')
FRED_API_KEY = os.getenv('FRED_API_KEY')

# Initialize Azure OpenAI
llm = AzureChatOpenAI(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    azure_deployment=AZURE_OPENAI_GPT_DEPLOYMENT_NAME,
    openai_api_version=AZURE_OPENAI_API_VERSION,
    openai_api_key=AZURE_OPENAI_API_KEY,
    temperature=0.7
)

# Initialize embeddings for vector database
embeddings = AzureOpenAIEmbeddings(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    azure_deployment=AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME,
    openai_api_version=AZURE_OPENAI_API_VERSION,
    openai_api_key=AZURE_OPENAI_API_KEY
)

print("Environment setup completed successfully!")

In [ ]:
# Swapnil
class PromptConfiguration:
    """Central configuration class for all prompts used in the investment research system"""
    
    @staticmethod
    def get_planning_prompt(role: str, task: str) -> str:
        """Get planning prompt for research tasks
        
        TODO: Create a formatted string prompt that:
        - Takes the agent's role (e.g., "Investment Analyst") and specific task
        - Guides the agent to create a detailed research plan
        - Should cover: data gathering, analysis techniques, risk assessment, market context
        - Returns a numbered list of specific, actionable research steps
        - Use f-string formatting: f"As an {role}, create a detailed research plan for: {task}"
        - Include sections for: data gathering, analysis techniques, risk assessment, market context
        """
        pass
    
    @staticmethod
    def get_reflection_prompt(analysis: str, context: str = "") -> str:
        """Get self-reflection prompt for quality assessment
        
        TODO: Create a prompt that evaluates analysis quality on:
        - Completeness (1-10): Are all key aspects covered?
        - Data Quality (1-10): Is the data comprehensive and current?
        - Logic (1-10): Is the reasoning sound and well-structured?
        - Actionability (1-10): Are the conclusions practical and specific?
        - Risk Assessment (1-10): Are risks properly identified and evaluated?
        - Should return JSON format with scores, strengths, improvements, recommendations
        """
        pass
    
    @staticmethod
    def get_news_classification_prompt(news_text: str) -> str:
        """Get news classification prompt for sentiment analysis
        
        TODO: Create a prompt that classifies news articles with:
        - Takes title and description as parameters
        - Returns JSON format with:
          - category: "earnings|product|market|regulation|management|merger|other"
          - sentiment: "positive|negative|neutral" 
          - importance: "high|medium|low"
          - reasoning: brief explanation
        """
        pass
    
    @staticmethod
    def get_news_extraction_prompt(news_data: List[Dict]) -> str:
        """Get news extraction prompt for key information
        
        TODO: Create a prompt that extracts insights from classified articles:
        - Takes classified articles as input
        - Returns JSON with: key_themes, sentiment_distribution, high_importance_items,
          potential_catalysts, risk_factors
        - Should analyze multiple articles and identify patterns
        """
        pass
    
    @staticmethod
    def get_news_summarization_prompt(extracted_data: str) -> str:
        """Get news summarization prompt
        
        TODO: Create comprehensive news analysis summary covering:
        - Executive Summary (2-3 sentences)
        - Key Developments and Themes
        - Sentiment Analysis
        - Potential Stock Price Catalysts
        - Risk Factors to Monitor
        - Investment Implications
        - Should be suitable for investment decision-making
        """
        pass
    
    @staticmethod
    def get_routing_prompt(request: str, symbol: str) -> str:
        """Get routing prompt for specialist assignment
        
        TODO: Create a prompt that determines which specialists to use:
        - Available specialists: technical, fundamental, news
        - Returns JSON with: specialists_needed, priority_order, reasoning
        - Should route based on request type and symbol needs
        """
        pass
    
    @staticmethod
    def get_technical_analysis_prompt(symbol: str, stock_data: str) -> str:
        """Get technical analysis prompt
        
        TODO: Create comprehensive technical analysis prompt covering:
        - Price Trend Analysis (short-term and medium-term)
        - Support and Resistance Levels
        - Volume Analysis, Key Technical Indicators, Chart Patterns
        - Technical Price Targets, Risk Levels and Stop-Loss Recommendations
        - Should conclude with: Technical Rating, Confidence Level, Key Risks, Price Levels to Watch
        """
        pass
    
    @staticmethod
    def get_fundamental_analysis_prompt(symbol: str, stock_data: str, alpha_overview: str) -> str:
        """Get fundamental analysis prompt
        
        TODO: Create comprehensive fundamental analysis covering:
        - Company Business Model and Competitive Position
        - Financial Health Assessment, Valuation Analysis (P/E, PEG ratios)
        - Growth Prospects, Management Quality, Industry Analysis
        - Competitive Advantages and Moats
        - Should conclude with: Fundamental Rating, Fair Value Estimate, Key Risks, Catalysts
        """
        pass
    
    @staticmethod
    def get_news_analysis_prompt(symbol: str, news_summary: str) -> str:
        """Get news analysis prompt
        
        TODO: Create sentiment analysis prompt focusing on:
        - Market Sentiment Implications
        - News Flow Impact on Stock Price
        - Institutional vs Retail Sentiment
        - Social Media and Public Perception Trends
        - News-Based Trading Opportunities, Event-Driven Catalysts
        - Should conclude with: Sentiment Rating, News Impact Assessment, Recommended Action
        """
        pass
    
    @staticmethod
    def get_evaluation_prompt(analysis: str, criteria: List[str]) -> str:
        """Get evaluation prompt for analysis quality assessment
        
        TODO: Create quality evaluator prompt that scores analysis on:
        - Completeness, Data Integration, Risk Assessment, Actionability
        - Logic and Reasoning, Market Context, Clarity (all 1-10 scale)
        - Returns JSON with: scores dict, overall_score, grade (A-F)
        - Include: strengths, weaknesses, specific_improvements, missing_elements
        """
        pass
    
    @staticmethod
    def get_optimization_prompt(analysis: str, evaluation: str, iteration: int) -> str:
        """Get optimization prompt for analysis refinement
        
        TODO: Create refinement prompt that:
        - Takes original analysis and evaluation feedback
        - Addresses identified weaknesses and adds missing elements
        - Implements specific improvements from evaluation
        - Focuses on areas that scored below 7/10
        - Maintains professional investment analysis standards
        """
        pass
    
    @staticmethod
    def get_synthesis_prompt(specialist_analyses: Dict[str, Any]) -> str:
        """Get synthesis prompt for combining specialist analyses
        
        TODO: Create comprehensive investment analysis prompt that:
        - Combines technical, fundamental, and news specialist reports
        - Creates structured analysis with: Executive Summary, Investment Thesis
        - Includes: Key Strengths/Opportunities, Risks/Concerns, Analysis Summaries
        - Concludes with: Price Target, Recommendation, Risk-Adjusted Returns, Action Items
        """
        pass

print("Prompt configuration class created!")

## 2. Data Source Tools

In [ ]:
# alphavantage_tool.py
import os
import requests
import json
from datetime import datetime
from IPython.display import display, JSON

# Set your API Key (o puedes guardarla en .env si prefieres)
os.environ["ALPHA_VANTAGE_API_KEY"] = "1PDRJTNTBLV913LG"

BASE_URL = "https://www.alphavantage.co/query"


def get_stock_data(symbol: str, function: str = "TIME_SERIES_DAILY", outputsize: str = "compact") -> dict:
    """
    Retrieve comprehensive stock data (price, volume, and basic metrics) from Alpha Vantage API.

    Parameters
    ----------
    symbol : str
        Stock ticker (e.g., 'AAPL', 'TSLA')
    function : str, optional
        API function ('TIME_SERIES_DAILY', 'TIME_SERIES_WEEKLY', 'TIME_SERIES_MONTHLY')
    outputsize : str, optional
        'compact' (last 100 points) or 'full' (20+ years of data)

    Returns
    -------
    dict
        Dictionary with metadata, latest price data, and computed metrics.
    """
    try:
        api_key = os.getenv("ALPHA_VANTAGE_API_KEY")
        if not api_key:
            return {"ok": False, "error": "Missing ALPHA_VANTAGE_API_KEY"}

        # --- API Request ---
        params = {
            "function": function,
            "symbol": symbol,
            "apikey": api_key,
            "outputsize": outputsize
        }
        response = requests.get(BASE_URL, params=params)
        response.raise_for_status()
        data = response.json()

        # --- Detect Time Series Key ---
        ts_key = next((k for k in data.keys() if "Time Series" in k), None)
        if not ts_key:
            return {"ok": False, "symbol": symbol, "error": "No time series data found"}

        # --- Extract Most Recent Entry ---
        timeseries = data[ts_key]
        latest_date = sorted(timeseries.keys())[-1]
        latest_data = timeseries[latest_date]

        open_price = float(latest_data["1. open"])
        high_price = float(latest_data["2. high"])
        low_price = float(latest_data["3. low"])
        close_price = float(latest_data["4. close"])
        volume = int(latest_data["5. volume"])

        # --- Compute Change ---
        change = close_price - open_price
        pct_change = (change / open_price) * 100

        result = {
            "ok": True,
            "symbol": symbol.upper(),
            "latest_date": latest_date,
            "open": round(open_price, 2),
            "high": round(high_price, 2),
            "low": round(low_price, 2),
            "close": round(close_price, 2),
            "volume": volume,
            "change": round(change, 2),
            "pct_change": round(pct_change, 2),
            "metadata": data.get("Meta Data", {})
        }

        return result

    except Exception as e:
        return {"ok": False, "symbol": symbol, "error": str(e)}


# ==============================================================
# Example
# ==============================================================

symbol = "AAPL"
result = get_stock_data(symbol)

# Results
display(JSON(result))

<IPython.core.display.JSON object>

In [ ]:
# News

import os
import json
import requests
from datetime import datetime, timedelta
from IPython.display import display, JSON

# Set your API Key
os.environ["NEWS_API_KEY"] = "23e94f19b6f844cdadcc235c4d327023"

NEWS_API_URL = "https://newsapi.org/v2/everything"


def get_stock_news(symbol: str, days: int = 7, max_results: int = 5) -> dict:
    """
    Get recent news articles for a stock symbol using NewsAPI.

    Parameters
    ----------
    symbol : str
        Stock ticker or company name (e.g., 'AAPL', 'Tesla')
    days : int, optional
        Number of days back to search (default: 7)
    max_results : int, optional
        Max number of articles to return (default: 5)

    Returns
    -------
    dict
        Dictionary with a list of top news articles.
    """
    try:
        api_key = os.getenv("NEWS_API_KEY")
        if not api_key:
            return {"ok": False, "error": "Missing NEWS_API_KEY"}

        from_date = (datetime.utcnow() - timedelta(days=days)).strftime("%Y-%m-%d")

        params = {
            "q": symbol,
            "from": from_date,
            "language": "en",
            "sortBy": "relevancy",
            "pageSize": max_results,
            "apiKey": api_key
        }

        response = requests.get(NEWS_API_URL, params=params)
        response.raise_for_status()
        data = response.json()

        if data.get("status") != "ok":
            return {"ok": False, "symbol": symbol, "error": data.get("message", "Unknown error")}

        articles = []
        for article in data.get("articles", []):
            articles.append({
                "title": article.get("title"),
                "description": article.get("description"),
                "source": article.get("source", {}).get("name"),
                "published_at": article.get("publishedAt"),
                "url": article.get("url")
            })

        return {
            "ok": True,
            "symbol": symbol.upper(),
            "from_date": from_date,
            "total_results": len(articles),
            "news": articles
        }

    except Exception as e:
        return {"ok": False, "symbol": symbol, "error": str(e)}


# ==============================================================
# Example
# ==============================================================

symbol = "AAPL"
result = get_stock_news(symbol, days=7)

display(JSON(result))

C:\Users\nelso\AppData\Local\Temp\ipykernel_15644\2096742089.py:36: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  from_date = (datetime.utcnow() - timedelta(days=days)).strftime("%Y-%m-%d")


<IPython.core.display.JSON object>

In [ ]:
# ==============================================================
# Economic Data Tool - Inline Implementation (FRED API)
# ==============================================================

import os
import json
from datetime import datetime
from fredapi import Fred
from IPython.display import display, JSON

# Set your API Key (ya incluida)
os.environ["FRED_API_KEY"] = "18ba88115a8a4249d92dd18b817b752f"

def get_economic_data(indicator: str = "GDP", start_date: str = "2020-01-01") -> dict:
    """
    Get economic indicators from FRED (Federal Reserve Economic Data).

    Parameters
    ----------
    indicator : str
        Series ID (e.g., 'GDP', 'UNRATE', 'FEDFUNDS', 'CPIAUCSL')
    start_date : str
        Earliest date (YYYY-MM-DD)

    Returns
    -------
    dict
        JSON with series_id, latest_value, latest_date, previous_value, change, change_pct
    """
    try:
        api_key = os.getenv("FRED_API_KEY")
        if not api_key:
            return {"ok": False, "error": "Missing FRED_API_KEY"}

        fred = Fred(api_key=api_key)

        # Get series data (limit 12 latest observations)
        data = fred.get_series(indicator, observation_start=start_date)

        if data is None or len(data) < 2:
            return {"ok": False, "indicator": indicator, "error": "Not enough data returned"}

        latest_date = data.index[-1].strftime("%Y-%m-%d")
        previous_date = data.index[-2].strftime("%Y-%m-%d")

        latest_value = float(data.iloc[-1])
        previous_value = float(data.iloc[-2])

        change = latest_value - previous_value
        change_pct = (change / previous_value) * 100 if previous_value != 0 else None

        result = {
            "ok": True,
            "indicator": indicator,
            "latest_date": latest_date,
            "latest_value": round(latest_value, 3),
            "previous_date": previous_date,
            "previous_value": round(previous_value, 3),
            "change": round(change, 3),
            "change_pct": round(change_pct, 2) if change_pct is not None else None
        }

        return result

    except Exception as e:
        return {"ok": False, "indicator": indicator, "error": str(e)}


# ==============================================================
# Example of use
# ==============================================================

indicator = "GDP"  # Puedes cambiar a 'UNRATE', 'FEDFUNDS', 'CPIAUCSL', etc.
result = get_economic_data(indicator, start_date="2020-01-01")

display(JSON(result))


<IPython.core.display.JSON object>

In [ ]:
#SEC

import requests
import pandas as pd
from bs4 import BeautifulSoup
from IPython.display import display, Markdown

def get_sec_financials(symbol="AAPL", form_type="10-K", limit=1):
    """
    Retrieve and display SEC filings (10-K / 10-Q) with extracted financial statements.
    Matches the display format of existing .ipynb notebooks.
    """
    try:
        cik_lookup = {
            "AAPL": "0000320193",
            "MSFT": "0000789019",
            "TSLA": "0001318605",
            "AMZN": "0001018724",
            "GOOG": "0001652044"
        }

        cik = cik_lookup.get(symbol.upper())
        if not cik:
            display(Markdown(f"❌ **No CIK found for symbol:** `{symbol}`"))
            return

        headers = {"User-Agent": "project@gmail.com"}
        url = f"https://data.sec.gov/submissions/CIK{cik.zfill(10)}.json"
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        data = response.json()

        filings = data.get("filings", {}).get("recent", {})
        results = []

        for i in range(len(filings.get("accessionNumber", []))):
            form_value = filings["form"][i].upper()
            if form_type in form_value:
                accession = filings["accessionNumber"][i].replace("-", "")
                doc_name = filings["primaryDocument"][i]
                filing_url = f"https://www.sec.gov/Archives/edgar/data/{int(cik)}/{accession}/{doc_name}"

                results.append({
                    "form_type": form_value,
                    "filed_at": filings["filingDate"][i],
                    "report_period": filings["reportDate"][i],
                    "accession": filings["accessionNumber"][i],
                    "primary_doc": doc_name,
                    "link": filing_url
                })

            if len(results) >= limit:
                break

        if not results:
            display(Markdown("❌ **No filings found.**"))
            return

        # --- Display header info ---
        latest = results[0]
        display(Markdown(f"### 🧾 {symbol.upper()} — {form_type} Filing"))
        display(Markdown(f"**Filed at:** {latest['filed_at']}  \n**Report period:** {latest['report_period']}  \n[🔗 View filing on SEC.gov]({latest['link']})"))

        # --- Extract HTML ---
        html = requests.get(latest["link"], headers=headers).text
        soup = BeautifulSoup(html, "html.parser")

        # --- Find and display financial tables ---
        tables = soup.find_all("table")
        found = 0

        for t in tables:
            text = t.get_text(" ", strip=True).lower()
            if any(x in text for x in ["balance sheet", "income statement", "cash flow", "operations", "financial position"]):
                df = pd.read_html(str(t))[0]
                found += 1
                display(Markdown(f"#### 📊 Statement {found}"))
                display(df)
                if found >= 3:
                    break

        if found == 0:
            display(Markdown("⚠️ **No structured financial tables found in this filing.**"))

    except Exception as e:
        display(Markdown(f"❌ **Error:** {str(e)}"))


# Example
get_sec_financials("AAPL", "10-K", 1)

### 🧾 AAPL — 10-K Filing

**Filed at:** 2024-11-01  
**Report period:** 2024-09-28  
[🔗 View filing on SEC.gov](https://www.sec.gov/Archives/edgar/data/320193/000032019324000123/aapl-20240928.htm)

C:\Users\nelso\AppData\Local\Temp\ipykernel_15644\3381631723.py:75: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(t))[0]


#### 📊 Statement 1

,0,1,2,3,4,5,6,7,8
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,Page,Page,Page
2,Part I,Part I,Part I,Part I,Part I,Part I,Part I,Part I,Part I
3,Item 1.,Item 1.,Item 1.,Business,Business,Business,1,1,1
4,Item 1A.,Item 1A.,Item 1A.,Risk Factors,Risk Factors,Risk Factors,5,5,5
5,Item 1B.,Item 1B.,Item 1B.,Unresolved Staff Comments,Unresolved Staff Comments,Unresolved Staff Comments,17,17,17
6,Item 1C.,Item 1C.,Item 1C.,Cybersecurity,Cybersecurity,Cybersecurity,17,17,17
7,Item 2.,Item 2.,Item 2.,Properties,Properties,Properties,18,18,18
8,Item 3.,Item 3.,Item 3.,Legal Proceedings,Legal Proceedings,Legal Proceedings,18,18,18
9,Item 4.,Item 4.,Item 4.,Mine Safety Disclosures,Mine Safety Disclosures,Mine Safety Disclosures,18,18,18


...

#### 📊 Statement 2

,0,1,2,3,4,5,6,7,8
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Index to Consolidated Financial Statements,Index to Consolidated Financial Statements,Index to Consolidated Financial Statements,NaN,NaN,NaN,Page,Page,Page
2,Consolidated Statements of Operations for the ...,Consolidated Statements of Operations for the ...,Consolidated Statements of Operations for the ...,NaN,NaN,NaN,29,29,29
3,Consolidated Statements of Comprehensive Incom...,Consolidated Statements of Comprehensive Incom...,Consolidated Statements of Comprehensive Incom...,NaN,NaN,NaN,30,30,30
4,Consolidated Balance Sheets as of September 28...,Consolidated Balance Sheets as of September 28...,Consolidated Balance Sheets as of September 28...,NaN,NaN,NaN,31,31,31
5,Consolidated Statements of Shareholders’ Equit...,Consolidated Statements of Shareholders’ Equit...,Consolidated Statements of Shareholders’ Equit...,NaN,NaN,NaN,32,32,32
6,Consolidated Statements of Cash Flows for the ...,Consolidated Statements of Cash Flows for the ...,Consolidated Statements of Cash Flows for the ...,NaN,NaN,NaN,33,33,33
7,Notes to Consolidated Financial Statements,Notes to Consolidated Financial Statements,Notes to Consolidated Financial Statements,NaN,NaN,NaN,34,34,34
8,Reports of Independent Registered Public Accou...,Reports of Independent Registered Public Accou...,Reports of Independent Registered Public Accou...,NaN,NaN,NaN,48,48,48


C:\Users\nelso\AppData\Local\Temp\ipykernel_15644\3381631723.py:75: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(t))[0]


#### 📊 Statement 3

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,Years ended,Years ended,Years ended,Years ended,Years ended,Years ended,Years ended,Years ended,Years ended,Years ended,Years ended,Years ended,Years ended,Years ended,Years ended
2,NaN,NaN,NaN,"September 28, 2024","September 28, 2024","September 28, 2024",NaN,NaN,NaN,"September 30, 2023","September 30, 2023","September 30, 2023",NaN,NaN,NaN,"September 24, 2022","September 24, 2022","September 24, 2022"
3,"Cash, cash equivalents, and restricted cash an...","Cash, cash equivalents, and restricted cash an...","Cash, cash equivalents, and restricted cash an...",$,30737,NaN,NaN,NaN,NaN,$,24977,NaN,NaN,NaN,NaN,$,35929,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Operating activities:,Operating activities:,Operating activities:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Net income,Net income,Net income,93736,93736,NaN,NaN,NaN,NaN,96995,96995,NaN,NaN,NaN,NaN,99803,99803,NaN
7,Adjustments to reconcile net income to cash ge...,Adjustments to reconcile net income to cash ge...,Adjustments to reconcile net income to cash ge...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Depreciation and amortization,Depreciation and amortization,Depreciation and amortization,11445,11445,NaN,NaN,NaN,NaN,11519,11519,NaN,NaN,NaN,NaN,11104,11104,NaN
9,Share-based compensation expense,Share-based compensation expense,Share-based compensation expense,11688,11688,NaN,NaN,NaN,NaN,10833,10833,NaN,NaN,NaN,NaN,9038,9038,NaN


In [ ]:
#Risk Factors / business model

import requests
from sec_api import ExtractorApi
from IPython.display import display, Markdown

API_KEY = "45b8d570f669460bf8c63ff405fd07d3b66e5c438e82849bdcb01ee9044cc699"
extractorApi = ExtractorApi(API_KEY)

def get_business_and_risk_factors(symbol="AAPL"):
    """
    Extracts both 'Business' (Item 1) and 'Risk Factors' (Item 1A)
    from the most recent 10-K filing.
    """

    try:
        # --- Obtener el CIK automáticamente ---
        headers = {"User-Agent": "project@gmail.com"}
        cik_url = f"https://www.sec.gov/cgi-bin/browse-edgar?CIK={symbol}&owner=exclude&action=getcompany"
        cik_resp = requests.get(cik_url, headers=headers)
        cik_resp.raise_for_status()

        import re
        match = re.search(r"CIK=(\d{10})", cik_resp.text)
        if not match:
            display(Markdown(f"❌ **No CIK found for `{symbol}`**"))
            return
        cik = match.group(1)

        # --- Obtener el último 10-K ---
        subm_url = f"https://data.sec.gov/submissions/CIK{cik}.json"
        data = requests.get(subm_url, headers=headers).json()
        filings = data.get("filings", {}).get("recent", {})
        filing_url = None

        for i in range(len(filings.get("accessionNumber", []))):
            if "10-K" in filings["form"][i].upper():
                acc = filings["accessionNumber"][i].replace("-", "")
                doc = filings["primaryDocument"][i]
                filing_url = f"https://www.sec.gov/Archives/edgar/data/{int(cik)}/{acc}/{doc}"
                break

        if not filing_url:
            display(Markdown("⚠️ **No 10-K filings found for this company.**"))
            return

        display(Markdown(f"=== Processing ticker: `{symbol}` ==="))
        display(Markdown(f"**Latest 10-K filing found:** [{filing_url}]({filing_url})"))

        # --- Extraer secciones ---
        business_text = extractorApi.get_section(filing_url, "1", "text")
        risk_text = extractorApi.get_section(filing_url, "1A", "text")

        # --- Mostrar resultados limpios ---
        display(Markdown("\n\n--- BUSINESS SECTION (ITEM 1) ---\n"))
        print(business_text[:5000], "...", "\n\n")  # muestra primeros 5000 caracteres

        display(Markdown("\n\n============================================================\n"))
        display(Markdown("\n\n--- RISK FACTORS SECTION (ITEM 1A) ---\n"))
        print(risk_text[:5000], "...")

    except Exception as e:
        display(Markdown(f"❌ **Error:** {str(e)}"))


# Example
get_business_and_risk_factors("AAPL")

=== Processing ticker: `AAPL` ===

**Latest 10-K filing found:** [https://www.sec.gov/Archives/edgar/data/320193/000032019324000123/aapl-20240928.htm](https://www.sec.gov/Archives/edgar/data/320193/000032019324000123/aapl-20240928.htm)



--- BUSINESS SECTION (ITEM 1) ---


 Item 1. Business 

Company Background 

The Company designs, manufactures and markets smartphones, personal computers, tablets, wearables and accessories, and sells a variety of related services. The Company&#8217;s fiscal year is the 52- or 53-week period that ends on the last Saturday of September. 

Products 

iPhone 

iPhone &#174; is the Company&#8217;s line of smartphones based on its iOS operating system. The iPhone line includes iPhone 16 Pro, iPhone 16, iPhone 15, iPhone 14 and iPhone SE &#174; . 

Mac 

Mac &#174; is the Company&#8217;s line of personal computers based on its macOS &#174; operating system. The Mac line includes laptops MacBook Air &#174; and MacBook Pro &#174; , as well as desktops iMac &#174; , Mac mini &#174; , Mac Studio &#174; and Mac Pro &#174; . 

iPad 

iPad &#174; is the Company&#8217;s line of multipurpose tablets based on its iPadOS &#174; operating system. The iPad line includes iPad Pro &#174; , iPad Air &#174; , iPad and iPad mini &#174; . 

Wea



============================================================




--- RISK FACTORS SECTION (ITEM 1A) ---


 Item 1A. Risk Factors 

The Company&#8217;s business, reputation, results of operations, financial condition and stock price can be affected by a number of factors, whether currently known or unknown, including those described below. When any one or more of these risks materialize from time to time, the Company&#8217;s business, reputation, results of operations, financial condition and stock price can be materially and adversely affected. 

Because of the following factors, as well as other factors affecting the Company&#8217;s results of operations and financial condition, past financial performance should not be considered to be a reliable indicator of future performance, and investors should not use historical trends to anticipate results or trends in future periods. This discussion of risk factors contains forward-looking statements. 

This section should be read in conjunction with Part II, Item 7, &#8220;Management&#8217;s Discussion and Analysis of Financial Condition and Resu

In [14]:
# Nelson
@tool
def get_stock_data(symbol: str, period: str = "1y") -> str:
    """Get comprehensive stock data including price, volume, and basic metrics.
    
    TODO: Implement using yfinance library:
    - Create yf.Ticker(symbol) object
    - Get historical data with stock.history(period=period)
    - Extract current_price, price_change, price_change_pct, volume
    - Get company info with stock.info for market_cap, pe_ratio, company_name, sector, industry
    - Return JSON string with all data formatted nicely
    - Handle exceptions and return error message if data retrieval fails
    """
    pass

@tool
def get_stock_news(symbol: str, days: int = 7) -> str:
    """Get recent news articles for a stock symbol.
    
    TODO: Implement using NewsApiClient:
    - Initialize NewsApiClient with NEWS_API_KEY
    - Calculate from_date using datetime.now() - timedelta(days=days)
    - Use newsapi.get_everything() with symbol as query, language='en', sort_by='relevancy'
    - Extract top 5 articles with: title, description, source, published_at, url
    - Return JSON string of news_items list
    - Handle exceptions and return error message if news retrieval fails
    """
    pass

@tool
def get_economic_data(indicator: str = "GDP", start_date: str = "2020-01-01") -> str:
    """Get economic indicators from FRED (Federal Reserve Economic Data).
    
    TODO: Implement using fredapi.Fred:
    - Initialize Fred(api_key=FRED_API_KEY)
    - Use fred.get_series(series_id, limit=12) to get last 12 observations
    - Calculate latest_value, previous_value, change, change_pct
    - Return JSON with series_id, latest_value, latest_date, previous_value, change, change_pct
    - Common series IDs: 'GDP', 'UNRATE', 'FEDFUNDS', 'CPIAUCSL'
    - Handle exceptions and return error message if economic data retrieval fails
    """
    pass

@tool
def get_alpha_vantage_data(symbol: str, function: str = "OVERVIEW") -> str:
    """Get detailed financial data from Alpha Vantage API.
    
    TODO: Implement using requests library:
    - Create base_url = "https://www.alphavantage.co/query"
    - Set params with function, symbol, and ALPHA_VANTAGE_API_KEY
    - Make GET request and parse JSON response
    - For OVERVIEW function: extract symbol, market_cap, pe_ratio, peg_ratio, dividend_yield, eps, 52_week_high, 52_week_low
    - Return JSON string (truncate to avoid token limits)
    - Handle exceptions and return error message if Alpha Vantage data retrieval fails
    """
    pass

print("Data source tools created!")

Data source tools created!


## 3. Agent Memory System

In [ ]:
# Chris
class TechnicalAnalyst:
    """Agent specialized in technical analysis of stock price and volume data.
    
    TODO: Implement technical analysis capabilities:
    - Accept price data (OHLCV) as input
    - Calculate key technical indicators: moving averages (SMA, EMA), RSI, MACD, Bollinger Bands
    - Identify chart patterns and trend analysis
    - Provide buy/sell/hold recommendations based on technical signals
    - Return structured analysis with indicator values, signals, and confidence levels
    """
    def __init__(self, llm, prompt_config):
        # TODO: Store LLM client and prompt configuration
        pass
    
    def analyze(self, stock_data, symbol):
        # TODO: Implement technical analysis logic:
        # - Parse stock_data JSON to extract price/volume information
        # - Use prompt_config.get_technical_analysis_prompt() for analysis template
        # - Send structured prompt to LLM with price data and technical requirements
        # - Return analysis with technical indicators, trend assessment, and trading signals
        pass

print("TechnicalAnalyst class created!")

## 4. Base Agent Class with Core Functions

In [ ]:
# Chris
class FundamentalAnalyst:
    """Agent specialized in fundamental analysis of company financials and valuation.
    
    TODO: Implement fundamental analysis capabilities:
    - Analyze financial metrics: P/E ratio, PEG ratio, dividend yield, market cap
    - Evaluate company fundamentals: revenue growth, profitability, debt levels
    - Compare metrics to industry averages and historical performance
    - Assess intrinsic value and investment quality
    - Return structured analysis with valuation metrics and investment thesis
    """
    def __init__(self, llm, prompt_config):
        # TODO: Store LLM client and prompt configuration
        pass
    
    def analyze(self, stock_data, alpha_vantage_data, symbol):
        # TODO: Implement fundamental analysis logic:
        # - Parse both stock_data and alpha_vantage_data for financial metrics
        # - Use prompt_config.get_fundamental_analysis_prompt() for analysis template
        # - Calculate and evaluate key ratios and financial health indicators
        # - Return analysis with valuation assessment, strengths/weaknesses, and investment rating
        pass

print("FundamentalAnalyst class created!")

## 5. Workflow Pattern 1: Prompt Chaining (News Processing Pipeline)

In [ ]:
# Chris
class NewsAnalyst:
    """Agent specialized in sentiment analysis and news impact assessment.
    
    TODO: Implement news sentiment analysis capabilities:
    - Process news articles for sentiment (positive/negative/neutral)
    - Identify key themes and market-moving events
    - Assess potential impact on stock price and investor sentiment
    - Correlate news sentiment with recent price movements
    - Return structured analysis with sentiment scores and impact assessment
    """
    def __init__(self, llm, prompt_config):
        # TODO: Store LLM client and prompt configuration
        pass
    
    def analyze(self, news_data, symbol):
        # TODO: Implement news sentiment analysis logic:
        # - Parse news_data JSON to extract article titles, descriptions, and sources
        # - Use prompt_config.get_news_analysis_prompt() for sentiment analysis template
        # - Analyze each article for sentiment and relevance to stock performance
        # - Aggregate sentiment scores and identify key themes/events
        # - Return analysis with overall sentiment, key insights, and potential price impact
        pass

print("NewsAnalyst class created!")

## 6. Workflow Pattern 2: Routing (Specialist Agents)

In [ ]:
# Nelson
class AgentMemory:
    """Persistent memory system for storing and retrieving agent experiences and insights.
    
    TODO: Implement FAISS-based vector memory system:
    - Initialize FAISS index for similarity search and storage
    - Use embeddings to store agent experiences, analysis results, and insights
    - Implement add_memory() to store new experiences with metadata (timestamp, symbol, analysis_type)
    - Implement search_memory() to retrieve relevant past experiences using similarity search
    - Implement save_memory() and load_memory() for persistence across sessions
    - Support different memory types: analysis_results, market_insights, user_preferences
    """
    
    def __init__(self, embedding_model):
        # TODO: Initialize FAISS index and embedding model
        # - Create FAISS IndexFlatL2 for vector similarity search
        # - Store embedding_model for converting text to vectors
        # - Initialize memories list for storing text and metadata
        # - Set memory_file path for persistence
        pass
    
    def add_memory(self, content, memory_type, symbol=None, metadata=None):
        # TODO: Add new memory to the vector store:
        # - Convert content to embedding using embedding_model
        # - Add vector to FAISS index
        # - Store memory with metadata (type, symbol, timestamp, etc.)
        # - Automatically save to disk for persistence
        pass
    
    def search_memory(self, query, memory_type=None, k=5):
        # TODO: Search for relevant memories:
        # - Convert query to embedding
        # - Use FAISS index.search() to find k most similar memories
        # - Filter by memory_type if specified
        # - Return list of relevant memories with similarity scores
        pass
    
    def save_memory(self):
        # TODO: Persist memory to disk:
        # - Save FAISS index using faiss.write_index()
        # - Save memories list and metadata to pickle file
        # - Handle exceptions and log save status
        pass
    
    def load_memory(self):
        # TODO: Load memory from disk:
        # - Load FAISS index using faiss.read_index()
        # - Load memories list and metadata from pickle file
        # - Handle file not found exceptions gracefully
        # - Return True if loaded successfully, False otherwise
        pass

print("AgentMemory class created!")

##6.5

In [6]:
# agent_memory.py
# FAISS-based Persistent Agent Memory + Tool Auto-Logging
# Requirements: faiss-cpu, numpy. For embeddings: any model exposing .encode(str) -> 1D np.ndarray

## 6.5 – agent_memory.py
# FAISS-based Persistent Agent Memory + Tool Auto-Logging
# Requirements: faiss-cpu, numpy, pickle
# Compatible with sentence-transformers or any model exposing .encode(str) -> np.ndarray

import os
import json
import time
import faiss
import pickle
import numpy as np
from datetime import datetime
from functools import wraps
from typing import Any, Callable, Dict, List, Optional, Tuple


# ---------- Embedding utilities ----------
class EmbeddingAdapter:
    """
    Minimal adapter for models exposing .encode(text) -> array-like
    (e.g., sentence_transformers, Instructor, etc.)
    """
    def __init__(self, model: Any):
        self.model = model

    def encode(self, text: str) -> np.ndarray:
        vec = self.model.encode(text)
        arr = np.asarray(vec, dtype="float32").reshape(-1)
        return arr


# ---------- FAISS-based persistent memory ----------
class AgentMemory:
    """Persistent memory system for storing and retrieving agent experiences and insights."""

    def __init__(self, embedding_model: EmbeddingAdapter, dim: int = 384):
        # Initialize FAISS index
        self.embedding_model = embedding_model
        self.index = faiss.IndexFlatL2(dim)
        self.memories: List[Dict[str, Any]] = []
        self.dim = dim
        print(f"✅ AgentMemory initialized with vector dimension {dim}")

    # ---------- Add memory ----------
    def add_memory(
        self,
        content: str,
        memory_type: str,
        symbol: Optional[str] = None,
        metadata: Optional[Dict[str, Any]] = None
    ):
        """Add a new memory to the FAISS index."""
        vec = self.embedding_model.encode(content).reshape(1, -1)
        self.index.add(vec)

        entry = {
            "content": content,
            "type": memory_type,
            "symbol": symbol,
            "metadata": metadata or {},
            "timestamp": datetime.now().isoformat(),
        }
        self.memories.append(entry)
        print(f"🧠 Added memory: {memory_type} ({symbol or 'no symbol'})")

    # ---------- Search memory ----------
    def search_memory(self, query: str, top_k: int = 5):
        """Retrieve top_k similar memories to a query using FAISS similarity search."""
        if len(self.memories) == 0:
            print("⚠️ No memories available to search.")
            return []

        qvec = self.embedding_model.encode(query).reshape(1, -1)
        D, I = self.index.search(qvec, top_k)
        results = []
        for i, dist in zip(I[0], D[0]):
            if 0 <= i < len(self.memories):
                mem = self.memories[i]
                results.append((mem["content"], float(dist), mem.get("metadata", {})))
        return results

    # ---------- Save memory ----------
    def save_memory(self, filepath: str = "agent_memory"):
        """Save FAISS index and memory metadata to disk."""
        faiss.write_index(self.index, f"{filepath}.index")
        with open(f"{filepath}.pkl", "wb") as f:
            pickle.dump(self.memories, f)
        print(f"💾 Memory saved to {filepath}.index / {filepath}.pkl")

    # ---------- Load memory ----------
    def load_memory(self, filepath: str = "agent_memory"):
        """Load FAISS index and memory metadata from disk."""
        self.index = faiss.read_index(f"{filepath}.index")
        with open(f"{filepath}.pkl", "rb") as f:
            self.memories = pickle.load(f)
        print(f"♻️ Memory loaded from {filepath}.index / {filepath}.pkl")

    # ---------- Decorator for auto-logging ----------
    def log_tool_usage(self, func: Callable):
        """Decorator that logs a tool's input/output into memory automatically."""
        @wraps(func)
        def wrapper(*args, **kwargs):
            start = time.time()
            result = func(*args, **kwargs)
            duration = time.time() - start

            log_entry = f"Tool {func.__name__} executed in {duration:.2f}s with args={args}, kwargs={kwargs}, result={str(result)[:300]}"
            self.add_memory(
                log_entry,
                memory_type="tool_usage",
                metadata={"duration": duration, "function": func.__name__}
            )
            return result
        return wrapper


In [7]:
## 6.6 – Testing AgentMemory Implementation (Fixed)
# Works regardless of search_memory() argument naming

import numpy as np
import inspect

# ---------- Dummy embedding model ----------
class DummyEmbeddingModel:
    """Mock embedding model that returns random vectors for testing."""
    def encode(self, text: str) -> np.ndarray:
        np.random.seed(abs(hash(text)) % (2**32))
        return np.random.rand(384)

# ---------- Initialize memory system ----------
embedding_model = EmbeddingAdapter(DummyEmbeddingModel())
memory = AgentMemory(embedding_model)

# ---------- Step 1: Add sample memories ----------
memory.add_memory(
    "Apple 10-K analysis: strong cash flow and expanding services segment",
    memory_type="analysis_results",
    symbol="AAPL",
    metadata={"source": "SEC", "date": "2025-10-11"}
)

memory.add_memory(
    "Tesla market insights: production ramping up and margin pressure decreasing",
    memory_type="market_insights",
    symbol="TSLA",
    metadata={"source": "NewsAPI", "date": "2025-10-11"}
)

memory.add_memory(
    "Microsoft investment report: AI integration boosting Azure growth",
    memory_type="analysis_results",
    symbol="MSFT",
    metadata={"source": "FRED", "date": "2025-10-11"}
)

print("✅ Added 3 memory entries.\n")

# ---------- Step 2: Perform similarity search ----------
query = "cash flow from Apple"

# Detect parameter name automatically
sig = inspect.signature(memory.search_memory)
params = list(sig.parameters.keys())
if len(params) > 1:
    # If function expects more than one argument (besides self)
    param_name = params[1]
    results = memory.search_memory(query, **{param_name: 2})
else:
    # If it only expects the query text
    results = memory.search_memory(query)

print(f"🔍 Query: {query}\n")
print("Top matches:")
for i, item in enumerate(results, 1):
    if isinstance(item, (list, tuple)) and len(item) >= 3:
        text, score, meta = item[:3]
        print(f"{i}. ({score:.4f}) {text} — {meta}")
    else:
        print(f"{i}. {item}")

# ---------- Step 3: Save memory ----------
save_path = "test_agent_memory.pkl"
memory.save_memory(save_path)
print(f"\n💾 Memory saved to {save_path}")

# ---------- Step 4: Reload memory ----------
new_memory = AgentMemory(embedding_model)
new_memory.load_memory(save_path)

print("\n♻️ Memory reloaded. Testing again...\n")

# Repeat query
if len(params) > 1:
    results_after_load = new_memory.search_memory("Azure growth", **{param_name: 1})
else:
    results_after_load = new_memory.search_memory("Azure growth")

print("Reloaded memory result:", results_after_load[0])


✅ AgentMemory initialized with vector dimension 384
🧠 Added memory: analysis_results (AAPL)
🧠 Added memory: market_insights (TSLA)
🧠 Added memory: analysis_results (MSFT)
✅ Added 3 memory entries.

🔍 Query: cash flow from Apple

Top matches:
1. (60.9385) Apple 10-K analysis: strong cash flow and expanding services segment — {'source': 'SEC', 'date': '2025-10-11'}
2. (65.0467) Microsoft investment report: AI integration boosting Azure growth — {'source': 'FRED', 'date': '2025-10-11'}
💾 Memory saved to test_agent_memory.pkl.index / test_agent_memory.pkl.pkl

💾 Memory saved to test_agent_memory.pkl
✅ AgentMemory initialized with vector dimension 384
♻️ Memory loaded from test_agent_memory.pkl.index / test_agent_memory.pkl.pkl

♻️ Memory reloaded. Testing again...

Reloaded memory result: ('Microsoft investment report: AI integration boosting Azure growth', 57.53370666503906, {'source': 'FRED', 'date': '2025-10-11'})


## 7. Workflow Pattern 3: Evaluator-Optimizer (Analysis Refinement Loop)

In [ ]:
# Swapnil
class NewsProcessingChain:
    """LangChain-based pipeline for processing and analyzing news data.
    
    TODO: Implement 5-step news processing pipeline:
    1. Ingest: Load and parse news articles from various sources
    2. Preprocess: Clean text, remove duplicates, standardize format
    3. Classify: Categorize news by relevance and impact level
    4. Extract: Identify key entities, events, and sentiment indicators
    5. Summarize: Generate concise summaries with investment implications
    """
    
    def __init__(self, llm):
        # TODO: Initialize LangChain components:
        # - Store LLM for processing steps
        # - Create chain components for each processing step
        # - Set up prompt templates for classification and extraction
        pass
    
    def process_news(self, news_data, symbol):
        # TODO: Execute complete news processing pipeline:
        # - Step 1: Parse news_data JSON and validate articles
        # - Step 2: Clean and deduplicate news articles
        # - Step 3: Classify articles by relevance to symbol and market impact
        # - Step 4: Extract entities, sentiment, and key events using LLM
        # - Step 5: Generate investment-focused summary of all relevant news
        # - Return structured result with processed articles and summary
        pass

print("NewsProcessingChain class created!")

## 8. Main Investment Research Agent Coordinator

In [ ]:
# Swapnil
class InvestmentResearchAgent:
    """Main coordinating agent that orchestrates the complete investment research process.
    
    TODO: Implement comprehensive investment research orchestration:
    - Coordinate data collection from all sources (stock, news, economic, Alpha Vantage)
    - Route analysis to appropriate specialist agents (technical, fundamental, news)
    - Implement planning phase to determine research approach
    - Implement reflection phase to validate and synthesize results  
    - Implement learning phase to update memory with insights
    - Generate final investment recommendation with confidence levels
    """
    
    def __init__(self, llm, memory, prompt_config):
        # TODO: Initialize main coordinating agent:
        # - Store LLM client, memory system, and prompt configuration
        # - Create instances of specialist agents (TechnicalAnalyst, FundamentalAnalyst, NewsAnalyst)
        # - Initialize NewsProcessingChain for news analysis
        # - Set up tools for data collection (get_stock_data, get_stock_news, etc.)
        pass
    
    def research_stock(self, symbol, analysis_type="comprehensive"):
        # TODO: Execute complete stock research workflow:
        # - Planning Phase: Use prompt_config to determine research approach
        # - Data Collection: Gather data from all relevant sources
        # - Specialist Analysis: Route to appropriate analysts based on analysis_type
        # - Synthesis: Combine all analysis results into coherent assessment
        # - Reflection Phase: Validate results and identify gaps
        # - Learning Phase: Store insights in memory for future use
        # - Return comprehensive investment report with recommendations
        pass

print("InvestmentResearchAgent class created!")

## 9. Visualization and Reporting Tools

In [ ]:
# Nelson
# TODO: Main execution workflow - Create and use the investment research system

# TODO: Initialize the system components:
# - Set up Azure OpenAI client with proper credentials
# - Create embedding model for memory system (e.g., text-embedding-ada-002)
# - Initialize AgentMemory with embedding model
# - Create PromptConfiguration instance for all prompt templates
# - Initialize InvestmentResearchAgent with LLM, memory, and prompts

# TODO: Example usage pattern:
# 1. Create investment_agent = InvestmentResearchAgent(llm, memory, prompt_config)
# 2. Load any existing memory: memory.load_memory()
# 3. Research a stock: result = investment_agent.research_stock("AAPL", "comprehensive")
# 4. Display results and save memory: memory.save_memory()

# TODO: Implement error handling and logging:
# - Wrap API calls in try-catch blocks
# - Log all research activities and results
# - Handle rate limiting and API failures gracefully
# - Provide fallback options when data sources are unavailable

print("Investment Research Agent System Ready!")
print("Usage: agent.research_stock('SYMBOL', analysis_type='comprehensive|technical|fundamental|news')")

## 10. Gradio Web Interface

In [ ]:
# Swapnil
# TODO: Testing and validation framework

# TODO: Implement comprehensive testing:
# - Test each data source tool individually (get_stock_data, get_stock_news, etc.)
# - Test specialist agents with sample data to verify analysis quality
# - Test memory system save/load functionality and search capabilities
# - Test complete research workflow with known stocks for validation
# - Implement unit tests for core functionality and error handling

# TODO: Example test cases:
# 1. Test data collection: verify all APIs return properly formatted data
# 2. Test analysis quality: compare agent outputs with expected results
# 3. Test memory persistence: save data, restart system, verify retrieval
# 4. Test error handling: simulate API failures and validate graceful handling
# 5. Test performance: measure response times and optimize bottlenecks

print("Testing framework ready - implement comprehensive validation before production use!")

## 11. Interface Launch

In [ ]:
# TODO: Configuration and deployment notes

# TODO: Required API keys and setup:
# - AZURE_OPENAI_API_KEY: Azure OpenAI service key
# - AZURE_OPENAI_ENDPOINT: Azure OpenAI service endpoint
# - NEWS_API_KEY: NewsAPI.org key for news data
# - FRED_API_KEY: Federal Reserve Economic Data API key  
# - ALPHA_VANTAGE_API_KEY: Alpha Vantage financial data API key

# TODO: Required Python packages:
# - langchain: For LLM orchestration and chaining
# - openai: Azure OpenAI client
# - yfinance: Yahoo Finance stock data
# - newsapi-python: News API client
# - fredapi: Federal Reserve Economic Data API
# - faiss-cpu: Vector similarity search for memory
# - pandas: Data manipulation and analysis
# - numpy: Numerical computations
# - requests: HTTP requests for Alpha Vantage

print("Configuration complete - ensure all API keys are properly set!")

## 12. Testing and Demo

In [ ]:
# TODO: Advanced features and extensions

# TODO: Potential enhancements for production deployment:
# - Real-time data streaming: WebSocket connections for live price updates
# - Portfolio management: Track multiple stocks and generate portfolio-level insights  
# - Risk management: Implement position sizing and risk assessment algorithms
# - Backtesting framework: Test strategies against historical data
# - Web interface: Create FastAPI/Streamlit dashboard for user interaction
# - Database integration: Store research results and user preferences in database
# - Advanced analytics: Machine learning models for price prediction and pattern recognition
# - Alert system: Automated notifications for significant market events or analysis updates
# - Multi-language support: Analyze international markets and foreign language news
# - Integration APIs: Connect with brokerage APIs for automated trading capabilities

# TODO: Monitoring and maintenance:
# - Implement comprehensive logging and monitoring
# - Set up automated testing and deployment pipelines
# - Monitor API usage and costs across all data sources
# - Regular model evaluation and prompt optimization
# - User feedback collection and analysis quality improvement

print("Advanced features planning complete - ready for production enhancements!")